In [3]:
import pandas as pd
import datetime as dt
import os

cwd = os.getcwd().rpartition("\\")[0]
adobe = cwd + r"\OneDrive - TIM\Team performance\Reports\SEO\Download Adobe\\"
semrush = cwd + r"\OneDrive - TIM\Team performance\Reports\SEO\Download Semrush\\"
fonte_dati_report = cwd + r"\OneDrive - TIM\Team performance\Reports\SEO\Fonte dati report\\"


#Adobe
traffic = pd.read_csv(adobe + r"Freeform table - Entry C44 - page url.csv", sep=",", header=[9,10])
traffic.columns = traffic.columns.droplevel()
traffic.columns = ["URL","PV Mobile","PV Desktop","Visits Mobile","Visits Desktop","Unique Visitors"]
traffic = traffic[["URL","PV Mobile","PV Desktop","Visits Mobile","Visits Desktop"]]
traffic["URL"] = traffic["URL"].str.split('?').str.get(0)
traffic["URL"] = traffic["URL"].str.split('#').str.get(0)
traffic = traffic.groupby("URL").sum().reset_index().sort_values(by="Visits Desktop", ascending=False)


In [5]:
#DESKTOP
pos = pd.read_excel(semrush +r"semrush_posizioni_DESKTOP.xlsx",
                    usecols=["Keyword", "Position", "Previous position", "Search Volume", "Keyword Difficulty", "CPC", "URL", "Traffic", "Traffic (%)", "Competition", "Timestamp", "SERP Features by Keyword"])

mese = pos["Timestamp"].max().strftime("%m")
anno = pos["Timestamp"].max().strftime("%Y")

pos["URL"] = pos["URL"].str.split('?').str.get(0)
pos["Timestamp"] = pos["Timestamp"].dt.date.max()
pos["Device"] = "Desktop"
pos = pos[(pos["Traffic"] > 100)]


pos_dk = pd.merge(pos,traffic, how="left", on="URL")
pos_dk.drop(columns=["PV Mobile","Visits Mobile"], inplace=True)
pos_dk.columns = ["Keyword", "Position", "Previous position", "Search Volume", "Keyword Difficulty", "CPC", "URL", "Traffic", "Traffic (%)", "Competition", "Timestamp", "SERP Features by Keyword","Device","Page Views", "Visits"]

#pos_dk.to_excel(fonte_dati_report + f"Mesi/posizioni_DESKTOP_{anno}{mese}.xlsx", index=False)

In [13]:
#MOBILE
pos_mob = pd.read_excel(semrush +r"semrush_posizioni_MOBILE.xlsx",
                    usecols=["Keyword", "Position", "Previous position", "Search Volume", "Keyword Difficulty", "CPC", "URL", "Traffic", "Traffic (%)", "Competition", "Timestamp", "SERP Features by Keyword"])

pos_mob["URL"] = pos_mob["URL"].str.split('?').str.get(0)
pos_mob["Timestamp"] = pos_mob["Timestamp"].dt.date.max()
pos_mob["Device"] = "Mobile"

pos_mob = pos_mob[(pos_mob["Traffic"] > 100)]

pos_mb = pd.merge(pos_mob,traffic, how="left", on="URL")
pos_mb.drop(columns=["PV Desktop","Visits Desktop"], inplace=True)
pos_mb.columns = ["Keyword", "Position", "Previous position", "Search Volume", "Keyword Difficulty", "CPC", "URL", "Traffic", "Traffic (%)", "Competition", "Timestamp", "SERP Features by Keyword","Device","Page Views", "Visits"]

#pos_mb.to_excel(fonte_dati_report + f"Mesi/posizioni_MOBILE_{anno}{mese}.xlsx", index=False)

In [16]:
traffic["Timestamp"] = pos["Timestamp"].max()
#traffic.loc[~(traffic["URL"] == "Entry C44 - page url")].to_excel(fonte_dati_report + f"Traffico/traffico_{anno}{mese}.xlsx", index=False)

In [30]:
#LIVELLI
liv_des = pos["URL"].str.replace("//","/").str.replace("/_","/").str.split('/', expand=True)
liv_des["URL"] = pos["URL"]
liv_des["URL"] = liv_des["URL"].str.split('?').str.get(0)
liv_des = liv_des[["URL", 2,3,4,5,6]]
liv_des.columns = ["URL","L1","L2","L3","L4","L5"]
liv_des.drop_duplicates(subset="URL", inplace=True)

liv_mob = pos_mob["URL"].str.replace("//","/").str.replace("/_","/").str.split('/', expand=True)
liv_mob["URL"] = pos_mob["URL"]
liv_mob["URL"] = liv_mob["URL"].str.split('?').str.get(0)
liv_mob = liv_mob[["URL", 2,3,4,5,6]]
liv_mob.columns = ["URL","L1","L2","L3","L4","L5"]
liv_mob.drop_duplicates(subset="URL", inplace=True)

liv_traffic = traffic["URL"].str.replace("//","/").str.replace("/_","/").str.split('/', expand=True)
liv_traffic["URL"] = traffic["URL"]
liv_traffic = liv_traffic[["URL", 2,3,4,5,6]]
liv_traffic.columns = ["URL","L1","L2","L3","L4","L5"]
liv_traffic.drop_duplicates(subset="URL", inplace=True)

liv = pd.concat([liv_des, liv_mob,liv_traffic], axis=0, ignore_index=True)
liv.sort_values(by="URL",inplace=True)
liv.drop_duplicates(subset="URL", keep="first", inplace= True)

#liv.to_excel(fonte_dati_report + f"Livelli/livelli_{anno}{mese}.xlsx", index=False)